### Load Packages

In [ ]:
import pandas as pd
import numpy as np
import os
import warnings

warnings.simplefilter("ignore")

pd.options.mode.chained_assignment = None

### Import Excel Files

In [ ]:
folder = r'C:\Users\mazhang\Documents\Jupyter Notebook\2021 Monthly Closed Invoice'
files = os.listdir(folder)

excel_list = []
for file in files:
    if file.endswith('.xlsx'):
        df = pd.read_excel(os.path.join(folder, file), sheet_name = None)
        excel_list.append(df)

### Generate Monthly Report with Accounting ID, Facility and  Bill Pay Category = Storage, etc.

In [ ]:
df_list = []
month = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep']

for excel in range(len(excel_list)):
    
    df_header = excel_list[excel]['Header']
    df_line = excel_list[excel]['Line']
    df_detail = excel_list[excel]['Detail']
 

    df_select_detail_int = df_detail[['CustomerID', 'DocNo', 'Facility',
                                      'Header Billing Period Start', 'Bill Pay Category']]
    df_select_detail = df_select_detail_int.loc[df_select_detail_int['Bill Pay Category'] == 'STORAGE INCOME']
       
    df_select_header = df_header[['Accounting_CustomerID', 'DocNo', 'Bill To']]
 

    df_combine_header_detail = pd.merge(df_select_detail, df_select_header, how = 'left',  on = ['DocNo']) 
    
    df_drop_dup_header = df_combine_header_detail.drop_duplicates()
    
    df_combine_rename = df_drop_dup_header.rename(columns = {'Header Billing Period Start':'Billing Period Start'})

    df_combine_month = df_combine_rename.reindex(columns = ['Accounting_CustomerID', 'CustomerID', 'Facility', 
                                                            'Bill Pay Category', 'Bill To', 'Billing Period Start',
                                                            'DocNo']).sort_values('Accounting_CustomerID')
    
    
    df_select_line_int = df_line[['CustomerID', 'Facility', 'Bill Pay Category', 'Bill To', 
                                  'DocNo', 'Cost']].sort_values('CustomerID')
    df_select_line = df_select_line_int.loc[df_select_line_int['Bill Pay Category'] == 'STORAGE INCOME']
    
    
    
    df1 = df_select_line.groupby(['CustomerID', 'Facility', 'Bill To'])['Cost'].sum().reset_index()
    
    df2 = df_combine_month.merge(df1, how = 'left', on = ['CustomerID', 'Facility', 'Bill To'])
    
    df3 = df2.drop(['Billing Period Start', 'DocNo'], axis = 1)

    df3 = df3.rename(columns = {'Cost': month[excel]})
    
    print(df3.shape)
    
    df_list.append(df3)

In [ ]:
df_combine_list = []
df_combine_quarter = df_list[0]

for df in range(1, len(df_list)):
    
    df_combine_quarter = pd.merge(df_combine_quarter, df_list[df], how = 'outer', 
                                  on = ['Accounting_CustomerID', 'CustomerID', 'Facility', 'Bill Pay Category', 'Bill To'])
    print(df_combine_quarter.shape)
    
    df_combine_list.append(df_combine_quarter)
    
df_output_init = df_combine_list[-1]
df_output = df_output_init.drop_duplicates().sort_values('Accounting_CustomerID')

In [ ]:
df_output

### Generate Monthly Report with Billing Count

In [ ]:
df_drop_iv = df_output.copy(deep = True)

df_count = df_drop_iv.drop_duplicates()
df_count['Count'] = df_count.drop(df_count.columns[0:5], axis = 1).count(axis = 1)
df_count = df_count.sort_values('Accounting_CustomerID')
df_count

In [ ]:
# df_count.to_excel("012021_092021 Monthly Report Storage Cost.xlsx", index = False, header = True)